In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

from summer.utils import ref_times_to_dti

from autumn.tools.plots.utils import REF_DATE
from autumn.tools.runs import ManagedRun
from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt
from autumn.settings.region import Region

## Specify the runs of interest

In [ ]:
run_id = "covid_19/myanmar/1639653326/62270c2"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
full_run = mr.full_run.get_derived_outputs()
pbi = mr.powerbi.get_db()
targets = pbi.get_targets()
results = pbi.get_uncertainty()
mcmc_params = mr.calibration.get_mcmc_params()

In [ ]:
mcmc_params.columns

In [ ]:
plt.style.use("ggplot")

param_lookup = {
    "contact_rate": "infection risk per contact",
    "infectious_seed": "starting infectious population",
    "mobility.microdistancing.behaviour.parameters.end_asymptote": "microdistancing effect",
    "sojourn.compartment_periods_calculated.active.total_period": "active disease duration",
    "sojourn.compartment_periods_calculated.exposed.total_period": "incubation period",
    "testing_to_detection.assumed_cdr_parameter": "CDR at 1/10,000 daily testing rate",
    "vaccination.vacc_part_effect_duration": "duration of partial vaccine immunity",
    "voc_emergence.delta.contact_rate_multiplier": "relative infectiousness Delta",
    "waning_immunity_duration": "duration of natural immunity",
}

unit_lookup = {
    "contact_rate": "probability",
    "infectious_seed": "persons",
    "mobility.microdistancing.behaviour.parameters.end_asymptote": "proportion",
    "sojourn.compartment_periods_calculated.active.total_period": "days",
    "sojourn.compartment_periods_calculated.exposed.total_period": "days",
    "testing_to_detection.assumed_cdr_parameter": "proportion",
    "vaccination.vacc_part_effect_duration": "days",
    "voc_emergence.delta.contact_rate_multiplier": "ratio",
    "waning_immunity_duration": "days",
}


fig, axes = plt.subplots(3, 3, figsize=(12, 12), sharey="all")
for i_ax, axis in enumerate(axes.reshape(-1)):
    param = mcmc_params.columns[i_ax]
    axis.hist(mcmc_params[param])
    axis.set_title(param_lookup[param])
    axis.set_xlabel(unit_lookup[param])
    if i_ax % 3 == 0:
        axis.set_ylabel("runs")   
fig.suptitle("parameter posterior histograms", fontsize=15, y=1)
fig.tight_layout()

In [ ]:
title_lookup = {
    "notifications": "daily notifications",
    "infection_deaths": "COVID-19-specific deaths",
    "hospital_admissions": "new daily hospital admissions",
    "icu_admissions": "new daily admissions to ICU",
    "proportion_seropositive": "proportion recovered from COVID-19",
    "incidence": "daily new infections",
    "prop_incidence_strain_delta": "proportion of cases due to Delta",
    "hospital_admissions": "daily hospital admissions",
    "hospital_occupancy": "total hospital beds",
    "icu_admissions": "daily ICU admissions",
    "icu_occupancy": "total ICU beds",
}

fig, axes = plt.subplots(3, 2, figsize=(12, 12), sharex="all")
plot_left_date = datetime.date(2020, 8, 1)
plot_right_date = datetime.date(2021, 11, 19)
outputs = ("notifications", "infection_deaths", "hospital_admissions", "icu_admissions", "proportion_seropositive", "incidence")
for i_ax, axis in enumerate(axes.reshape(-1)):
    output = outputs[i_ax]
    scenario_chain = (full_run["scenario"] == 0) & (full_run["chain"] == 0)
    for i_run in full_run[scenario_chain]["run"].unique():
        selection = full_run[(full_run["run"] == i_run) & scenario_chain]
        axis.plot(ref_times_to_dti(REF_DATE, selection["times"]), selection[output])
    if output in targets:
        axis.scatter(targets.index, targets[output], color="k", s=10, alpha=0.5, zorder=10)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(title_lookup[output])
    axis.set_xlim(left=plot_left_date, right=plot_right_date)

In [ ]:
def plot_outputs(outputs, left_date, right_date):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex="all")
    colour = (0.2, 0.2, 0.8)
    for i_ax, axis in enumerate(axes.reshape(-1)):
        output = outputs[i_ax]
        results_df = results[(output, 0)]
        indices = results_df.index
        axis.fill_between(indices, results_df[0.025], results_df[0.975], color=colour, alpha=0.5)
        axis.fill_between(indices, results_df[0.25], results_df[0.75], color=colour, alpha=0.7)
        axis.plot(indices, results_df[0.500], color="b")
        if output in targets:
            axis.scatter(targets.index, targets[output], color="k", alpha=0.5, s=10)
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=left_date, right=right_date)
        axis.set_title(title_lookup[output])
    fig.tight_layout()

In [ ]:
left_date = datetime.date(2021, 9, 1)
right_date = datetime.date(2022, 12, 31)
outputs_to_plot = (
    "notifications", 
    "infection_deaths", 
    "proportion_seropositive", 
    "prop_incidence_strain_delta",
)
plot_outputs(outputs_to_plot, left_date, right_date)

In [ ]:
left_date = datetime.date(2021, 9, 1)
right_date = datetime.date(2022, 12, 31)
outputs_to_plot = (
    "hospital_admissions", 
    "hospital_occupancy", 
    "icu_admissions", 
    "icu_occupancy",
)
plot_outputs(outputs_to_plot, left_date, right_date)